<a href="https://colab.research.google.com/github/NINJAHATTORI004/programmingwithdataanalysis/blob/main/Autoencoders%2BDimensionality%2BReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Input,Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file=r'/content/drive/MyDrive/data/Pokemon.csv'
data=pd.read_csv(file)
data.head()

,ID,Name,Form,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,1,Bulbasaur,,Grass,Poison,318,45,49,49,65,65,45,1
1,2,Ivysaur,,Grass,Poison,405,60,62,63,80,80,60,1
2,3,Venusaur,,Grass,Poison,525,80,82,83,100,100,80,1
3,4,Charmander,,Fire,,309,39,52,43,60,50,65,1
4,5,Charmeleon,,Fire,,405,58,64,58,80,65,80,1


In [6]:
data['Generation'].value_counts(normalize=True)

,proportion
Generation,
5,0.135802
1,0.124280
8,0.120988
3,0.116049
9,0.115226
6,0.107819
7,0.100412
4,0.097119
2,0.082305


In [7]:
data['Generation'].dtype

dtype('int64')

In [8]:
data.nunique()

,0
ID,1025
Name,1026
Form,207
Type1,18
Type2,19
Total,228
HP,109
Attack,126
Defense,115
Sp. Atk,127


In [9]:
data.drop(['Name','Total'],axis=1, inplace=True)

In [10]:
data.isnull().sum()

,0
ID,0
Form,0
Type1,0
Type2,0
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0
Speed,0


In [11]:
data['Type 2'].fillna('missing',inplace=True)

KeyError: 'Type 2'

In [12]:
data['Generation']=data['Generation'].astype(int)

In [13]:
data1=data.copy()

In [14]:
cat_cols=['Type 1','Type 2','Generation']
for col in cat_cols:
  freqs=data1[col].value_counts()
  cats=freqs.index[freqs>20]
  for cat in cats:
    name=str(col)+'_'+str(cat)
    data1[name]=(data1[col]==cat).astype(int)
  del data1[col]

KeyError: 'Type 1'

In [ ]:
data1.head()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Type 1_Water,Type 1_Normal,...,Type 2_Fighting,Type 2_Grass,Type 2_Fairy,Type 2_Steel,Generation_1,Generation_5,Generation_3,Generation_4,Generation_2,Generation_6
0,1,45,49,49,65,65,45,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,60,62,63,80,80,60,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,80,82,83,100,100,80,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,80,100,123,122,120,80,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,39,52,43,60,50,65,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
data.shape

(800, 11)

In [ ]:
data1.shape

(800, 39)

In [ ]:
x=data1.drop('Legendary',axis=1)
y=data1.Legendary
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)
scores=cross_val_score (rf,x,y, scoring='roc_auc', cv=5)

In [ ]:
scores

array([0.94139194, 0.8822606 , 0.85609628, 0.87179487, 0.97043433])

In [ ]:

dummy_data=pd.get_dummies (data['Type 1'], prefix='Type1')
dummy_data.shape[1]

18

In [ ]:
y_type2=dummy_data

embedding_dim=3
inputs=Input(shape=(dummy_data.shape[1],))
dense1=Dense(15,activation='relu')(inputs)
embedded_output=Dense(embedding_dim)(dense1)

In [ ]:
outputs=Dense(dummy_data.shape[1], activation='softmax') (embedded_output)
model=Model(inputs=inputs, outputs=outputs)

In [ ]:
embedder=Model(inputs=inputs, outputs=embedded_output)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(dummy_data,y_type2,epochs=200,batch_size=100)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.0087 - loss: 2.8960    
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1396 - loss: 2.8757  
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1433 - loss: 2.8544 
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1555 - loss: 2.8299 
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1335 - loss: 2.8141 
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1337 - loss: 2.7942 
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1508 - loss: 2.7648 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1587 - loss: 2.7451 
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1399 - loss: 2.7291 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1385 - loss: 2.7031 
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1429 - loss: 2.6788 
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1425 -

In [ ]:
low_dim_type1=pd.DataFrame(embedder.predict(dummy_data),columns=['t1_emb1','t1_emb2','t1_emb3'])
low_dim_type2=pd.DataFrame(embedder.predict(dummy_data),columns=['t2_emb1','t2_emb2','t2_emb3'])
low_dim_type2.head()

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,t2_emb1,t2_emb2,t2_emb3
0,-3.799379,-7.561825,-5.186865
1,-3.799379,-7.561825,-5.186865
2,-3.799379,-7.561825,-5.186865
3,-3.799379,-7.561825,-5.186865
4,10.393406,-0.688957,-2.288917


In [ ]:
data2=pd.concat([low_dim_type1,low_dim_type2,data.select_dtypes(['int64']),pd.get_dummies (data['Generation'], prefix='Generation')], axis=1)

In [ ]:
scores.mean()

np.float64(0.9043956043956044)

In [ ]:
x=data2.drop('Legendary',axis=1)
y=data2['Legendary']
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)
scores=cross_val_score(rf,x,y,scoring='roc_auc',cv=5)
scores.mean()

NameError: name 'data2' is not defined